<a href="https://colab.research.google.com/github/kyunghyuncho/bio-ret-viz/blob/master/pyserini_scibert_covid19_demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Pyserini Demo on COVID-19 Dataset (Title + Abstract Index) 
# with HuggingFace Transformers based visualization


This notebook provides a demo on how to get started in searching the [COVID-19 Open Research Dataset](https://pages.semanticscholar.org/coronavirus-research) (release of 2020/03/20) from AI2.
In this notebook, we'll be working with the title + abstract index. 
Specifically, we're not indexing the full text (that'll come later, soon!).


In [0]:
from IPython.core.display import display, HTML

First, install Python dependencies

In [0]:
%%capture
!pip install pyserini==0.8.1.0
!pip install transformers

import json
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"

Let's grab the pre-built index:

In [0]:
%%capture
!wget https://www.dropbox.com/s/uvjwgy4re2myq5s/lucene-index-covid-2020-03-20.tar.gz
!tar xvfz lucene-index-covid-2020-03-20.tar.gz

In [74]:
!du -h lucene-index-covid-2020-03-20

1.3G	lucene-index-covid-2020-03-20


Let's load SciBERT (https://github.com/allenai/scibert) from HuggingFace Transformers

In [0]:
import torch
import numpy

In [0]:
from tqdm import tqdm

In [72]:
from transformers import *

tokenizer = AutoTokenizer.from_pretrained('allenai/scibert_scivocab_cased')
model = AutoModel.from_pretrained('allenai/scibert_scivocab_cased')

Sanity check of index size (should be 1.3G):

You can use `pysearch` to search over an index. Here's the basic usage:

In [0]:
query = 'integration degradation synthesis host immune'

In [113]:
from pyserini.search import pysearch

searcher = pysearch.SimpleSearcher('lucene-index-covid-2020-03-20/')
hits = searcher.search(query)

display(HTML('<div style="font-family: Times New Roman; font-size: 20px; padding-bottom:12px"><b>Query</b>: '+query+'</div>'))

# Prints the first 10 hits
for i in range(0, 10):
  display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:10px">' + 
               F'{i+1} {hits[i].docid} ({hits[i].score:1.2f}) -- ' +
               F'{hits[i].lucene_document.get("authors")} et al. ' +
              #  F'{hits[i].lucene_document.get("journal")}. ' +
              #  F'{hits[i].lucene_document.get("publish_time")}. ' +
               F'{hits[i].lucene_document.get("title")}. ' +
               F'<a href="https://doi.org/{hits[i].lucene_document.get("doi")}">{hits[i].lucene_document.get("doi")}</a>.'
               + '</div>'))

From the hits array, use `.lucene_document` to access the underlying indexed Lucene `Document`, and from there, call `.get(field)` to fetch specific fields, like "title", "doc", etc.
The complete list of available fields is [here](https://github.com/castorini/anserini/blob/master/src/main/java/io/anserini/index/generator/CovidGenerator.java#L46).

For hit #1, we don't have the full text, but we can access available information via `.raw`.

In [114]:
hit1_json = json.loads(hits[0].raw)
print(json.dumps(hit1_json, indent=4))

{
    "abstract": "The severe acute respiratory syndrome coronavirus (SARS-CoV) nsp1 protein has unique biological functions that have not been described in the viral proteins of any RNA viruses; expressed SARS-CoV nsp1 protein has been found to suppress host gene expression by promoting host mRNA degradation and inhibiting translation. We generated an nsp1 mutant (nsp1-mt) that neither promoted host mRNA degradation nor suppressed host protein synthesis in expressing cells. Both a SARS-CoV mutant virus, encoding the nsp1-mt protein (SARS-CoV-mt), and a wild-type virus (SARS-CoV-WT) replicated efficiently and exhibited similar one-step growth kinetics in susceptible cells. Both viruses accumulated similar amounts of virus-specific mRNAs and nsp1 protein in infected cells, whereas the amounts of endogenous host mRNAs were clearly higher in SARS-CoV-mt-infected cells than in SARS-CoV-WT-infected cells, in both the presence and absence of actinomycin D. Further, SARS-CoV-WT replication st

For hit #8, we have the full text, which we can also fetch via `.raw`: As an example, let's look at to which section the second paragraph belongs.

In [115]:
hit8_json = json.loads(hits[7].raw)
print(hit8_json['body_text'][1]['section'])

Introduction


Let's extract contextualized vectors of queries and abstracts from SciBERT for highlighting relevant paragraphs.

First, extract the contextualized vectors of the query above:

$$q_1, \ldots, q_T = \text{SciBERT}(\text{query})$$

In [0]:
def extract_scibert(text, tokenizer, model):
  # we consider only the first 512 tokens (for now)
  text = text[:512]

  text_ids = torch.tensor([tokenizer.encode(text, add_special_tokens=True)])
  text_words = tokenizer.convert_ids_to_tokens(text_ids[0])[1:-1]

  with torch.no_grad():
    state = model(text_ids)[0]
    state = state[:, 1:-1, :]

  return text_ids, text_words, state[0]

In [0]:
query_ids, query_words, query_state = extract_scibert(query, tokenizer, model)

Second, let's extract contextualized vectors of all the paragraphs from the hit #3:

$$p_1^k, \ldots, p_{T_k}^k = \text{SciBERT}(\text{paragraph}^k)$$

In [128]:
ii = 5

doc_json = json.loads(hits[ii].raw)

paragraph_states = []
for pid in tqdm(range(len(doc_json['body_text']))):
  paragraph_states.append(extract_scibert(doc_json['body_text'][pid]['text'], tokenizer, model))

100%|██████████| 102/102 [00:34<00:00,  2.99it/s]


We then compute the cosine similarity matrix between the query and each paragraph:

$$A^k = [a^k_{ij}] \in \mathbb{R}^{|\text{query}| \times |\text{paragraph}^k|},$$

where

$$a^k_{ij} = \frac{q_i^\top p_j^k}{\| q_i \| \| p_j^k \|}$$


In [0]:
def cross_match(state1, state2):
  state1 = state1 / torch.sqrt((state1 ** 2).sum(1, keepdims=True))
  state2 = state2 / torch.sqrt((state2 ** 2).sum(1, keepdims=True))

  sim = (state1.unsqueeze(1) * state2.unsqueeze(0)).sum(-1)

  return sim

In [130]:
sim_matrices = []

for pid in tqdm(range(len(doc_json['body_text']))):
  sim_matrices.append(cross_match(query_state, paragraph_states[pid][-1]))

100%|██████████| 102/102 [00:00<00:00, 584.45it/s]


Let's retrieve the most relevant paragraphs first, where define the top-$M$ most relevant paragraphs as 

$$\arg\text{top-$M$}_{k=1}^K \max_{i=1,\ldots,|\text{query}|} \max_{j=1,\ldots, |\text{paragraph}^k|} A_{ij}^k$$

that is, a paragraph with the highly matched words to the query words is considered relevant.

In [0]:
paragraph_relevance = [torch.max(sim).item() for sim in sim_matrices]
rel_index = numpy.argsort(paragraph_relevance)[-5:][::-1]

In [134]:
display(HTML('<div style="font-family: Times New Roman; font-size: 20px; padding-bottom:12px"><b>Query</b>: '+query+'</div>'))

display(HTML('<div style="font-family: Times New Roman; font-size: 20px; padding-bottom:12px"><b>Document</b>: '+
             F'{hits[i].lucene_document.get("authors")} et al. ' +
             F'{hits[i].lucene_document.get("title")}. ' + 
             F'<a href="https://doi.org/{hits[i].lucene_document.get("doi")}">{hits[i].lucene_document.get("doi")}</a>.' +
             '</div>'))

for ri in numpy.sort(rel_index):
  display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:10px; margin-left: 15px">' + 
               F'<b>{doc_json["body_text"][ri]["section"]}</b> -- ' +
               F'{" ".join(paragraph_states[ri][1]).replace(" ##","")}' + ' ... </div>'))

We want to look at more details by highlighting relevant phrases in each paragraph, where we define relevant phrases for each paragraph as

$$\arg\text{top-$M$}_{j=1,\ldots, |\text{paragraph}^k|} \max_{i=1,\ldots,|\text{query}|} A_{ij}^k$$

that is, any word that had a high similarity to each of the query words is considered relevant. given these words, we highlight a window of 10 surrounding each of them.

In [135]:
display(HTML('<div style="font-family: Times New Roman; font-size: 20px; padding-bottom:12px"><b>Query</b>: '+query+'</div>'))

display(HTML('<div style="font-family: Times New Roman; font-size: 20px; padding-bottom:12px"><b>Document</b>: '+
             F'{hits[i].lucene_document.get("authors")} et al. ' +
             F'{hits[i].lucene_document.get("title")}. ' + 
             F'<a href="https://doi.org/{hits[i].lucene_document.get("doi")}">{hits[i].lucene_document.get("doi")}</a>.' +
             '</div>'))

for ri in numpy.sort(rel_index):
  sim = sim_matrices[ri].data.numpy()
  rel_words = numpy.sort(numpy.argsort(sim.max(0))[-5:][::-1])

  ptext = paragraph_states[ri][1]

  para = ""
  prev_idx = 0
  for jj in rel_words:
    para = para + " "
    para = para + " ".join(ptext[prev_idx:jj-5])
    para = para + " <font color='red'>"
    para = para + " ".join(ptext[jj-5:jj+5])
    para = para + "</font> "
    prev_idx = jj+5

  display(HTML('<div style="font-family: Times New Roman; font-size: 18px; padding-bottom:10px; margin-left: 15px">' + 
               F'<b>{doc_json["body_text"][ri]["section"]}</b> -- ' +
               F'{para.replace(" ##","")}' + ' ... </div>'))